In [1]:
from adjacency import Adjacency, generate_random_genome
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
from torch.optim import lr_scheduler

code for dataset

for this dataset, the implemented model failed to obtain non-zero accuracy. In order to accurately reproduce the genome from the adjacency matrix, a considerable number of kmer is required, due to which the training data simply does not fit into the memory

In [2]:
# def read_fasta_file(file_path):
#     sequences = {}  # Dictionary to store sequences
#     with open(file_path, 'r') as f:
#         current_header = None
#         current_sequence = ''
#         for line in f:
#             line = line.strip()  # Remove leading/trailing whitespace
#             if line.startswith('>'):
#                 # If the line starts with '>', it's a header line
#                 # Store the previous sequence (if any) and update current header
#                 if current_header is not None:
#                     sequences[current_header] = current_sequence
#                 current_header = line[1:]  # Remove '>'
#                 current_sequence = ''
#             else:
#                 # If the line doesn't start with '>', it's sequence data
#                 current_sequence += line
#         # Store the last sequence
#         sequences[current_header] = current_sequence
#     return sequences

# file_path = 'genome.txt' 
# sequences = read_fasta_file(file_path)

# random.seed(42)

# # Function to replace 'N' with a random letter from 'ACGT'
# def replace_N_with_random_letter(string):
#     return ''.join(random.choice('ACGT') if char == 'N' else char for char in string)

# # Update dictionary values
# for key, value in sequences.items():
#     sequences[key] = replace_N_with_random_letter(value)

# genomes = list(sequences.values())

randomly generated dataset

In [3]:
genomes = [generate_random_genome(5) for i in range(1000)]

converting genomes into adjacency matrices and testing accuracy genome -> matrix -> genome

In [4]:
k_mer_len = 3
matrix_size = 4**k_mer_len
adj = Adjacency(k_mer_len)
adj_genomes = [adj.genome2adjacency(i) for i in genomes]
genomes_adj = [adj.adjacency2genome(i[0], i[1]) for i in adj_genomes]

In [5]:
errors=0
for i in range(1000):
    if genomes[i] != genomes_adj[i]:
        errors +=1
print(f"errors: {errors}/{1000}")

errors: 13/1000


model consists of encoder and decoder with 3 linear layers in each. I used MSE loss function and rmsprop optimizer, also tried adam and adagrad, but they did no better.

I read a little about how people trained encoders for sparse matrices, they used different regularization methods, custom loss functions, etc. https://www.quora.com/When-training-an-autoencoder-on-very-sparse-data-how-do-you-force-the-decoder-to-reconstruct-mostly-zeros-rather-than-always-just-reconstructing-the-average. but I did not find any meaningful code for that

There is an article https://arxiv.org/pdf/1905.03375 where the authors made an autoencoder for sparse matrices for recommender systems, in the abstract it was said that their simple linear model has better results than complex nonlinear ones. there is a code on github, I think it will be possible to try to figure out how to apply it to our data and see what happens

I have found that the following things help to increase accuracy: 

removing ReLUs between hidden layers

decreasing learning rate

the shorter the length of the genome, the greater the accuracy



For the genome with length 5 the following results were observed:

with RelUs, 3-mers acc = 0.3

with RelUs, 2-mers acc = 0.11

with RelUs, 1-mers acc = 0.03

without ReLUs between hidden layers, 3-mers acc = 0.34

removed last linear layer from encoder and first from decoder, without ReLUs between hidden layers, 3-mers acc = 0.27


since the accuracy on the training data is quite low, I didn't make train/test split. I calculated the accuracy by comparing the adjacency matrices.

In [6]:
class MyDataset(Dataset):
    def __init__(self, matrices):
        self.matrices = matrices

    def __len__(self):
        return len(self.matrices)

    def __getitem__(self, idx):
        return torch.tensor(self.matrices[idx], dtype=torch.float32)

In [7]:
class Autoencoder(nn.Module):
    def __init__(self, matrix_size):
        super(Autoencoder, self).__init__()
        self.matrix_size = matrix_size
        self.encoder = nn.Sequential(
            nn.Linear(matrix_size*matrix_size, matrix_size*4),  
            # nn.ReLU(),
            nn.Linear(matrix_size*4, matrix_size),  
            # nn.ReLU(),
            # nn.Linear(matrix_size, matrix_size//2),  
            # nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            # nn.Linear(matrix_size//2, matrix_size),  
            # nn.ReLU(),
            nn.Linear(matrix_size, matrix_size*4),  
            # nn.ReLU(),
            nn.Linear(matrix_size*4, matrix_size*matrix_size),
            nn.ReLU()  
        )

    def forward(self, x):
        x = x.view(-1, self.matrix_size*self.matrix_size)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(-1, self.matrix_size, self.matrix_size)
        return x

Here i tried to normalize matrices, but it had no effect

In [8]:
# matrices_normalized = [(matrix[0] - np.min(matrix[0])) / (np.max(matrix[0]) - np.min(matrix[0])) for matrix in adj_genomes]
matrices_normalized = [matrix[0] for matrix in adj_genomes]

Also tried lr scheduler, it didn't help

In [9]:
model = Autoencoder(matrix_size)

criterion = nn.MSELoss()

optimizer = optim.RMSprop(model.parameters(), lr=0.00001, alpha=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# optimizer = optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-4)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)
dataset = MyDataset(matrices_normalized)
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
testloader = DataLoader(dataset, batch_size=1, shuffle=False)


/home/maksym/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


training

In [10]:
num_epochs = 10000
for epoch in range(num_epochs):
    running_loss = 0.0
    for data in dataloader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, data)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    
    # Print loss every 20 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")
        
        # Evaluation
        all_predictions = []
        model.eval()
        for data in testloader:
            with torch.no_grad():
                outputs = model(data)
            all_predictions.append(outputs.numpy()[0])
        int_predictions = [np.round(matrix).astype(int) for matrix in all_predictions]
        errors = 0
        for i in range(len(matrices_normalized)):
            if not np.array_equal(int_predictions[i], matrices_normalized[i]):
                errors += 1
        print(f"Accuracy: {len(matrices_normalized)-errors}/{len(matrices_normalized)}")
    # scheduler.step()
        # print(f"Error Rate: {errors / len(matrices_normalized)}")


Epoch 100, Loss: 0.0004754977024276741
Accuracy: 0/1000
Epoch 200, Loss: 0.0004429528024047613
Accuracy: 0/1000
Epoch 300, Loss: 0.0003862620840664022
Accuracy: 12/1000
Epoch 400, Loss: 0.0003225998720154166
Accuracy: 109/1000
Epoch 500, Loss: 0.00027580811220104804
Accuracy: 194/1000
Epoch 600, Loss: 0.00025307465839432555
Accuracy: 246/1000
Epoch 700, Loss: 0.0002446484912070446
Accuracy: 267/1000
Epoch 800, Loss: 0.00024200209372793324
Accuracy: 267/1000
Epoch 900, Loss: 0.00024153721678885632
Accuracy: 268/1000
Epoch 1000, Loss: 0.00024146393770934081
Accuracy: 268/1000
Epoch 1100, Loss: 0.00024145515635609627
Accuracy: 268/1000
Epoch 1200, Loss: 0.000241455074865371
Accuracy: 268/1000
Epoch 1300, Loss: 0.0002414550785033498
Accuracy: 268/1000
Epoch 1400, Loss: 0.0002414550763205625
Accuracy: 268/1000
Epoch 1500, Loss: 0.00024145507995854132
Accuracy: 268/1000
Epoch 1600, Loss: 0.00024145507559296674
Accuracy: 268/1000
Epoch 1700, Loss: 0.0002414550763205625
Accuracy: 268/1000
Epoc

KeyboardInterrupt: 